# Primer modelo entrenado

## Descripción del proyecto

El objetivo del proyecto es desarrollar un modelo con el mayor grado de exactitud de la compañía móvil Megaline, la cuál no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Se requiere un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Se va a trabajar con el dataset `datasets/users_behavior.csv`, el cual contiene la información de los clientes. Este DataSet ya tiene procesados los datos. por lo que se pasará directo a la creación del modelo

## 1. Lectura de los datos

Se hará la importación de las librerías correspondientes para trabajar en el proyecto y se hará la lectura del dataset.

También se estudiará un poco el DataFrame para saber que columnas contiene y cuántos datos hay

In [18]:
# Importación de librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Lectura del dataset
df = pd.read_csv('datasets/users_behavior.csv')

In [3]:
display(df.head(5))
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Ahora sabemos que el DataFrame tiene 5 columnas y la columna `is_ultra` será nuestra columna objetivo, la cuál sólo contiene ceros y unos, que marcan el plan para el mes actual (Ultra - 1, Smart - 0). Esto convierte nuestra tarea en una tarea de <u>Clasificación Binaria</u>

## 2. Segmentación de los datos

Para poder entrenar el modelo debemos separar el dataset en tres conjuntos: entrenamiento, prueba y validación. Para saber cuántos datos debe contener cada set, tomando en cuenta que no se tiene un conjunto de prueba, se usará una porporción de datos de 3:1:1. Esto quiere decir que el 60% del dataset será para el entrenamiento, 20% para validación y 20% para prueba.

Pero, primero debemos separar las características del objetivo en diferentes conjuntos de nuestro dataset principal:

In [4]:
# df con las características
features = df.drop('is_ultra', axis=1)
display(features.sample(2))

# Objetivo
target = df['is_ultra']
display(target.sample(2))

,calls,minutes,messages,mb_used
1202,49.0,316.69,46.0,17519.29
911,73.0,463.74,29.0,21860.13


2587    1
1583    0
Name: is_ultra, dtype: int64

Una vez que se tienen las características y el objetivo por separado, se hará uso de la función `train_test_split` de la librería de `sklearn`. Esta función sólo puede divir el dataset en 2, por lo que se usará dos veces. Primero se separará el conjunto de entrenamiento y prueba, después se volverá a usar para dividir de nuevo el conjunto de entrenamiento en validación y entrenamiento.

In [5]:
# División del dataset en df_train y df_test
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state=12345)

In [6]:
# Segunda división del dataset
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size = 0.25, random_state=12345)
# Se usa 0.25 porque 0.8*0.25=0.2

## 3. Modelos

Se entrenarán tres modelos: árbol de decisión, bsque aleatorio y regresión logística. De estos se tomará el que mantenga un umbral de exactitud de 0.75 para comprobar la calidad con el conjunto de prueba.

### 3.1. Árbol de decisión

Para este modelo se investigará la calidad modificando el hiperparámetro `max_depth` para saber a qué profundidad de árbol es mejor la exactitud

In [12]:
# Se entrena el modelo para diferentes profundidades y se saca su exactitud
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("max_depth", depth, ":", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth 1 :0.7387247278382582
max_depth 2 :0.7573872472783826
max_depth 3 :0.7651632970451011
max_depth 4 :0.7636080870917574
max_depth 5 :0.7589424572317263


Al probar diferentes profundidades, se observa que la profundidad que más se acerca al umbral que queremos es la profundidad 2.

### 3.2. Bosque Aleatorio

Con este modelo se modificará el hiperparámetro `n_estimators` para saber cuál es la mejor cantidad de árboles para nuestro modelo. Se evaluarán hasta un máximo de 20 árboles debido a los procesos necesarios de computación

In [17]:
# Primero inicializamos nuestras variables que contendran el mejor score de la mejor estimación
best_score = 0
best_est = 0

# Se entrena el modelo para diferentes profundidades y se saca su score
for est in range (1,21):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train)
    score = model_forest.score(features_valid, target_valid)
    best_score = score
    best_est = est    
    print('La exactitud del modelo (n_estimators={}): {}'.format(best_est, best_score))

La exactitud del modelo (n_estimators=1): 0.702954898911353
La exactitud del modelo (n_estimators=2): 0.7573872472783826
La exactitud del modelo (n_estimators=3): 0.744945567651633
La exactitud del modelo (n_estimators=4): 0.7651632970451011
La exactitud del modelo (n_estimators=5): 0.7620528771384136
La exactitud del modelo (n_estimators=6): 0.7698289269051322
La exactitud del modelo (n_estimators=7): 0.7713841368584758
La exactitud del modelo (n_estimators=8): 0.7869362363919129
La exactitud del modelo (n_estimators=9): 0.7838258164852255
La exactitud del modelo (n_estimators=10): 0.7884914463452566
La exactitud del modelo (n_estimators=11): 0.7807153965785381
La exactitud del modelo (n_estimators=12): 0.7822706065318819
La exactitud del modelo (n_estimators=13): 0.7776049766718507
La exactitud del modelo (n_estimators=14): 0.7853810264385692
La exactitud del modelo (n_estimators=15): 0.7838258164852255
La exactitud del modelo (n_estimators=16): 0.7838258164852255
La exactitud del mo

Podemos observar entre más número de árboles hay, es mejor la exactitud. Sin embargo, nuestro umbral debe ser de 0.75 y el estimador que más se acerca a ese número es `n_estimators` = 3

### 3.3. Regresión logística

En este modelo vamos a especificar `solver` como `liblinear` porque es la más general y funciona bien con conjuntos de datos pequeños.

In [20]:
model_regression = LogisticRegression(random_state=12345, solver='liblinear')
model_regression.fit(features_train, target_train)
score_valid = model_regression.score(features_valid, target_valid)
print('La exactitud del modelo de Regresión Logística es:', score_valid)

La exactitud del modelo de Regresión Logística es: 0.7293934681181959


La exactitud no se acerca tanto al umbral que requerimos como los dos modelos anteriores.

Conclusiones:

El mejor modelo para poder probar la calidad con el conjunto de prueba es el bosque aleatorio con un estimador = 3

## 4. Modelo con conjunto de prueba

Se va a evaluar la calidad del conjunto de prueba con el mejor modelo que se encontró: Bosque aleatorio

In [21]:
# Se coloca n_estimators en 3
model_test = RandomForestClassifier(random_state=12345, n_estimators=3)
model_test.fit(features_train, target_train)
score_test = model_test.score(features_test, target_test)
print('La exactitud con el conjunto de prueba es de:', score_test)

La exactitud con el conjunto de prueba es de: 0.7729393468118196


Conclusiones:

La exactitud es de 0.77, la cuál se acerca al umbral que teníamos. Esta exactitud irá creciendo si se usan más números de árboles.

## 5. Prueba de cordura

Para esta prueba de cordura se ha decidido colocar todas las respuestas con el mismo valor, y verificar su exactitud. Se pondrá 0 para todas las respuestas 

In [24]:
# Se colocan todas las respuestas en 0
predictions = pd.Series(0, index=target.index)

# Se calcula la exactitud de predictions con el target
result = accuracy_score(target, predictions)
print('La exactitud entre el target y las predicciones puestas en 0 es de:', result)

La exactitud entre el target y las predicciones puestas en 0 es de: 0.693528313627878


Conclusion

La exactitud es de 0.69, la cuál si es baja, pero parece aceptable para un modelo cuyas predicciones son todas 0. Esto quiere decir que hay un 69% de probabilidad de acertar que el plan que elegirá el cliente, si colocamos que todos lo clientes quieren un plan Smart.

Nuestro modelo tiene una exactitud de 0.77. Esto quiere decir que nuetro modelo tiene una mejor probabilidad de poder predcir el plan que el cliente prefiere, ya sea Smart o Ultra. Nuestro modelo es mucho mejor.

Si nuestro modelo estuviera por debajo del 0.69, daría a entender que el modelo es tan malo que sería mejor no hacerlo y colocar que todos los clientes prefieren el plan Smart.